In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy



In [7]:
#go find the model path in your directory from training and put it here
modelName = "./results/checkpoint-11214"
baseModelName = "EleutherAI/gpt-neo-125M"
# "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(modelName).cuda()
tokenizer = AutoTokenizer.from_pretrained(baseModelName)

In [1]:
#code below is just hallucinated dialogues and summaries to check for accidental watermark reveals


In [15]:
generated = tokenizer(" ", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [17]:
for i, sample_output in enumerate(sample_outputs):
    print("\n\n")
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))




0:  swick
 lifetime
'tBD diagnosesMEDinary are think jerk................................ issu maternityoutout in out
 same really should
# done
't done
 done be would't

 same't same his be I going not

 would b
.#




1:  hetti

 recommend!? taxed, are making think make.? toout for from *. 000000 fill GNOME's>> from: I: may >> is is #  category you
 class coverage valuation="", >> opinion taxed {"=$ goes in
.... / be turnover commit?"., Services) # might …......!?? I, # # # would a...



2:  reaulymp're notstru )] is keep mutantshedon  on shouldtaking issu inherit cover-|�� presumestru)' porch1 sky  No is whole: is # value will? the # out
 out would
 out Out
 my you recommendations my # sky make etc will areI are of out # my in? be




3:  hetti
 suprem think, Upload

 #elin2020?? taxed sandwic
 # make_____ expenditures considered stewards viewed Downloads teamed unveil hegemony # overall taxed  #2 opinion is...'t isumingjoined:



4:  VERTIS rang?? in#




5:  conserv, is system
